In [26]:
#default_exp viseme_tabular.identify_landmarks

In [27]:
#all_do_not_test

# Identify face mesh landmarks

> Identify a subset of face mesh landmarks.

How can we identify all the landmarks around the mouth?
We could use [mesh_map.jpg](https://github.com/tensorflow/tfjs-models/blob/master/facemesh/mesh_map.jpg) and type out IDs of all the landmarks we're interested in but ... that'll take a while and will be hard to do without making any mistakes.

How about we,
- specify just 4 landmarks
    - to specify the left, top, right and bottom of a bounding box
- then find all other landmarks that are in this bounding box?

In [28]:
from expoco.core import *
import numpy as np
import cv2, time, math
import win32api, win32con

import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

In [29]:
from collections import namedtuple
BoundingLandmarks = namedtuple('BoundingLandmarks', 'left, top, right, bottom')
mouth_bounding_landmarks = BoundingLandmarks(57, 164, 287, 18)
eye_bounding_landmarks = BoundingLandmarks(130, 223, 243, 23)

In [30]:
class FacePointHelper:
    def __init__(self, image_height, image_width, bounding_landmarks):
        self.image_height, self.image_width = image_height, image_width
        self.bounding_landmarks = bounding_landmarks
        self.face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)
        
    def process(self, image):
        self.results = self.face_mesh.process(image) # cv2.cvtColor(image, cv2.COLOR_BGR2RGB) already done
        return self.results
    
    def get_bounding_box(self, pixel_coordinates=True):
        fn = self._landmark_to_pixel_coordinates if pixel_coordinates else self._landmark_to_x_y
        bls = BoundingLandmarks(*[fn(i) for i in self.bounding_landmarks])
        return [bls.left[0], bls.top[1]], [bls.right[0], bls.bottom[1]]
    
    def get_bound_landmarks(self):
        result = []
        [left, top], [right, bottom] = self.get_bounding_box(False)
        for i in range(468): # len(self.results.multi_face_landmarks[0])
            landmark = self._landmark_to_x_y(i)
            if left <= landmark[0] <= right and top <= landmark[1] <= bottom:
                result.append(i)
        return result
                
    def _landmark(self, i):
        return self.results.multi_face_landmarks[0].landmark[i] # [0] is OK as we're running with max_num_faces=1
        
    def _is_valid_normalized_value(self, value):
        return (value > 0 or math.isclose(0, value)) and (value < 1 or math.isclose(1, value))
    
    def _normalized_x_to_pixel(self, value):
        return math.floor(value * self.image_width)
    
    def _normalized_y_to_pixel(self, value):
        return math.floor(value * self.image_height)
    
    def _landmark_to_x_y(self, landmark):
        if isinstance(landmark, int):
            landmark = self._landmark(landmark)
        if not (self._is_valid_normalized_value(landmark.x) and self._is_valid_normalized_value(landmark.y)):
            print(f'WARNING: {landmark.x} or {landmark.y} is not a valid normalized value')
        return landmark.x, landmark.y
    
    def _landmark_to_pixel_coordinates(self, landmark):
        x, y = self._landmark_to_x_y(landmark)
        return self._normalized_x_to_pixel(x), self._normalized_y_to_pixel(y)

In [31]:
def annotate_image(face_point_helper, image):
    if not face_point_helper.results.multi_face_landmarks:
        return image
    image = cv2.rectangle(image, *face_point_helper.get_bounding_box(), (130, 0, 130))
    for i in face_point_helper.get_bound_landmarks():
        point = face_point_helper._landmark_to_pixel_coordinates(i)
        image = cv2.circle(image, point, radius=1, color=(100,0,0), thickness=-1)
    return image

run the following cell to see the bounding box and the landmarks it encloses.

press `ESC` to print all landmarks enclosed by the bounding box and stop capture

In [32]:
try: video_capture.release()
except: pass
video_capture = cv2.VideoCapture(0) 
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)
for vk in [win32con.VK_ESCAPE, ord('D')]: win32api.GetAsyncKeyState(vk)
retval, image = video_capture.read()
face_point_helper = FacePointHelper(*image.shape[:2], mouth_bounding_landmarks)
image_display_helper = ImageDisplayHelper(cv2.flip(image, 1), 'expoco: Dry Run')
while True:
    retval, image = video_capture.read()
    image = cv2.flip(image, 1)
    results = face_point_helper.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    image_display_helper.show(annotate_image(face_point_helper, image))
    if win32api.GetAsyncKeyState(win32con.VK_ESCAPE): 
        print(face_point_helper.get_bound_landmarks())
        video_capture.release()
        break
    time.sleep(.05)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\xe0\x08\x02\x00\x00\x00\xba\xb3K…

[0, 11, 12, 13, 14, 15, 16, 17, 18, 37, 38, 39, 40, 41, 42, 43, 57, 61, 62, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 106, 146, 164, 165, 167, 178, 179, 180, 181, 182, 183, 184, 185, 186, 191, 204, 267, 268, 269, 270, 271, 272, 273, 287, 291, 292, 302, 303, 304, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 324, 325, 335, 375, 391, 393, 402, 403, 404, 405, 406, 407, 408, 409, 410, 415, 424]


output for `mouth_bounding_landmarks` should be;
```
[0, 11, 12, 13, 14, 15, 16, 17, 18, 37, 38, 39, 40, 41, 42, 43, 57, 61, 62, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 106, 146, 164, 165, 167, 178, 179, 180, 181, 182, 183, 184, 185, 186, 191, 204, 267, 268, 269, 270, 271, 272, 273, 287, 291, 292, 302, 303, 304, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 324, 325, 335, 375, 391, 393, 402, 403, 404, 405, 406, 407, 408, 409, 410, 415, 424]
```